# Dimensional Analysis - The Quantity Class

Physical equations usually involve quantities with both a value and units. The units represent the variable's "dimension" in space, time and mass.

Dimensional analysis involves making sure that the dimensions of quantities in an equation are commensurate when combined. That is when two quantities are added, subtracted or equated they have the same dimension.

Dimensional analysis by-hand is often a pain, especially when variables are raised by exponents and expressed in different, often non-base, units. To prevent headaches I wrote the `Quantity` class to represent a value and its units.


#### References

https://en.wikipedia.org/wiki/Dimensional_analysis

https://en.wikipedia.org/wiki/SI_derived_unit


#### Similar packages

SymPy https://docs.sympy.org/latest/modules/physics/units/index.html

SymDim

Pint

In [1]:
from quantity import Quantity

import numpy as np

## Specifying a quantity

A `Quantity` is defined in terms of a value and a variable number of unit arguments. 

```python
my_quantity = Quantity(value, *units)
```

Each unit argument is either a single value or a tuple of up to three values in the following order:

``` python
unit = scale | (n, scale, p) | (scale, p) | (n, scale)
```

***n*** is a base-10 multiplier. This can be given as an integer (-6 for micro, -2 for centi, etc.) or short string ('mu' for micro-, 'c' for centi-, etc.). Allowable values are given in the table below.

***scale*** The unit name ('g' for gram, 'm' for metre, etc.). Any character or string is allowed.

***p*** - the dimensional exponent of the unit.

$$ unit = (10^n \times scale)^{p} $$


*n* and *p* an be exlcuded when they are superfluous ($n=0$ or $p=1$). For instance, the following declarations are equivalent for 3.5 grams.

In [2]:
mass = Quantity(3.5, (0, "g", 1))
print(mass)

mass = Quantity(3.5, (0, "g"))
print(mass)

mass = Quantity(3.5, ("g", 1))
print(mass)

mass = Quantity(3.5, "g")
print(mass)

3.5 g^1
3.5 g^1
3.5 g^1
3.5 g^1


These are string arguments for *n*.


| prefix | *n* | string arg |
|---|---|---|
|atto|-18|a|
|femto|-15|f|
|pico|-12|p|
|nano|-9|n|
|micro|-6|mu|
|milli|-3|m|
|centi|-2|c|
|deci|-1|d|
|deca|1|da|
|hecto|2|2|
|kilo|3|k|
|mega|6|M|
|giga|9|G|
|tera|12|T|
|peta|15|P|

For instance, the following are equivalent.

In [3]:
mass = Quantity(1.0, ('k', 'g'))
print(mass)

mass = Quantity(1.0, (3, 'g'))
print(mass)

1.0 kg^1
1.0 (10^3.g)^1


Quantities with compound units are declared with multiple unit arguments.

In [4]:
acceleration_of_freefall = Quantity(9.8, "m", ("s", -2))
print(acceleration_of_freefall)

resistance = Quantity(2.1, ("k" "g"), ("m", 2), ("s", -3), ("A", 2))
print(resistance)

9.8 m^1.s^-2
2.1 kg^1.m^2.s^-3.A^2


Unitless quantities have no unit arguments.

In [26]:
pi = Quantity(3.14159265)
print(pi)

3.14159265 


The value can be a numpy array.

In [5]:
speeds = Quantity(np.array([0.0, 0.5, 1.5, 5.6]), "m", ("s", -1))
print(speeds)

[0.  0.5 1.5 5.6] m^1.s^-1


## Base and derived units

Base SI units are specified by the following strings.

|SI unit|unit string|
|--|--|
|kilogram|kg|
|meter|m|
|second|s|
|Ampere|A|
|Kelvin|K|

A number of derived (non-base) SI units are also recognised.

|Category|derived unit|unit string|
|--|--|--|
|**Mechanical**|Joule|J|
||Newton|N|
||Watt|W|
||Pascal|Pa|
|**Electrical**|Ohm|Ohm|
||Siemen|S|
||Volt|V|
||Farad|F|
||Coulomb|C|
|**Non-compound**|Litre|L|
||Hertz|Hz|
||gram|g|


Quantities can be converted to base units by calling `to_base_units()`. However this isn't necessary other then for display.

In [6]:
pressure = Quantity(1.0, "Pa")
print(pressure.to_base_units())

resistance = Quantity(1.0, "Ohm")
print(resistance.to_base_units())

volume = Quantity(2.0, "L")
print(volume.to_base_units())

1.0 kg^1.m^-1.s^-2
1.0 kg^1.m^2.s^-3.A^-2
0.002 m^3


## Arithmetic & Commensurability

Quantaties can be added, subtracted, multiplied, divided and raised by an exponent.

In [10]:
mass = Quantity(2.0, "kg") + Quantity(50.0, "g")
print(mass)

mass = Quantity(2.0, "kg") - Quantity(50.0, "g")
print(mass)

mass_sq = Quantity(2.0, "kg") * Quantity(50.0, "g")
print(mass_sq)

mass_ratio = Quantity(2.0, "kg") / Quantity(50.0, "g")
print(mass_ratio)

mass_sq = Quantity(2.0, "kg")**2
print(mass_sq)

mass = Quantity(4.0, ("kg", 2))**0.5
print(mass)

2.05 kg^1
1.95 kg^1
0.1 kg^2
40.0 
4.0 kg^2
2.0 kg^1.0


Two quantities can also be tested for commensurability (i.e. do they have the same dimension?) with the equality operator.

In [8]:
print(Quantity(3.5, ("m", "L")) == Quantity(2.1, ("m", 3)))

print(Quantity(5.6, (-7, "N")) == Quantity(2.1, "kg", "m", ("s", -2)))

print(Quantity(3.5, ("m", "L")) == Quantity(5.6, (-7, "N")))

True
True
False


If you try to add or subtract incommensurable quantaties, an error is thrown.

In [13]:
Quantity(3.5, ("m", "L")) + Quantity(5.6, (-7, "N"))

TypeError: Cannot add 3.5 mL^1 and 5.6 (10^-7.N)^1.

In [14]:
Quantity(2.5, "kg").log10()

0.3979400086720376 kg^1

## Some more examples

A mechanical damped spring.

In [48]:
mass = Quantity(0.1, "kg")
spring_constant = Quantity(7.5, "kg", ("s", -2))
damping = Quantity(1.12, "kg", ("s", -1))

angular_frequency = (spring_constant / mass)**0.5
print(angular_frequency)

damping_coefficient = Quantity(0.5) * damping / (mass * spring_constant)**0.5
print(damping_coefficient)

8.660254037844387 s^-1.0
0.6466323014923809 


And its electrical (RLC circuit) analog.

In [47]:
inductance = Quantity(0.1, "kg", ("m", 2), ("s", -2), ("A", -2))
capacitance = Quantity(1/7.5, "F")
resistance = Quantity(1.12, "Ohm")

angular_frequency = (capacitance * inductance)**-0.5
print(angular_frequency)

damping_coefficient = Quantity(0.5) * resistance / (inductance / capacitance)**0.5
print(damping_coefficient)

8.660254037844386 s^-1.0
0.6466323014923809 
